In [0]:
import requests as req
import pandas as pd
import json
from pprint import pprint
from pyspark.sql import functions as f
from pyspark.sql import types as t

In [0]:
context = json.loads(dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson())
instancename = context['tags']['browserHostName']
token = dbutils.widgets._entry_point.getDbutils().notebook().getContext().apiToken().get()

In [0]:
job_lst=[]

headers = {
    'Authorization': f'Bearer {token}',
}
query_params = {
  'limit': 25,
  'offset': 0,
  'expand_tasks': 'true'
}
response = req.get(f"https://{instancename}/api/2.1/jobs/list", headers=headers, params=query_params).json()

has_more = response['has_more']

job_lst = job_lst+response['jobs']

while has_more:
  query_params['offset'] = query_params['offset'] + query_params['limit']
  recurse_response = req.get(f"https://{instancename}/api/2.1/jobs/list", headers=headers, params=query_params).json()
  job_lst = job_lst+recurse_response['jobs']
  has_more = recurse_response['has_more']
print("Job List count:", len(job_lst))  

Job List count: 498


In [0]:
def get_notebook_path(dic_obj):
  if 'notebook_task' in dic_obj.keys():
    notebook_path = dic_obj['notebook_task'].get('notebook_path')
    source = dic_obj['notebook_task'].get('source')
    return {'notebook_path':notebook_path, 'source':source}
  else:
    return {'notebook_path':"No notebooks tasks here", 'source':"No notebooks tasks here"}
    

In [0]:
parsed_job_lst = [(i.get('creator_user_name','Not available'),
                   i['job_id'],
                   any( [True for data in i['settings']['tasks'] if 'sql_task' in data.keys()]),
                   [get_notebook_path(data) for data in i['settings']['tasks'] if 'notebook_task' in data.keys() ]) for i in job_lst]

In [0]:
# data_schema = t.StructType([
#   t.StructField('creator_user_name',t.StringType()),
#   t.StructField('job_id',t.LongType()),
#   t.StructField('contains_sql_task',t.BooleanType()),
#   t.StructField('notebook_tasks',t.ArrayType(t.MapType(t.StringType(),t.StringType())))
# ])
data_schema = t.StructType([
  t.StructField('creator_user_name',t.StringType()),
  t.StructField('job_id',t.LongType()),
  t.StructField('contains_sql_task',t.BooleanType()),
  t.StructField("notebook_tasks", t.ArrayType(
      t.StructType([
          t.StructField("notebook_path", t.StringType()),
          t.StructField("source", t.StringType())
      ])
   )
  )
])
job_data = spark.createDataFrame(parsed_job_lst,data_schema)

In [0]:
exploded_job_data = job_data.withColumn('notebook_tasks',f.explode_outer('notebook_tasks'))
cleaned_job_data = exploded_job_data.select('creator_user_name','job_id','contains_sql_task',f.col('notebook_tasks.notebook_path').alias('notebook_path'),f.col('notebook_tasks.source').alias('source'))

In [0]:
@f.udf('string')
def workload_type(notebook_path,source):
  lang = None
  if source == 'WORKSPACE':
    auth_header = {
      'Authorization': f'Bearer {token}',
    }
    req_data = {
      'path': notebook_path
    }
    get_resp = req.get(f"https://{instancename}/api/2.0/workspace/list", headers=auth_header, params=req_data).json()
    if 'objects' in get_resp.keys():
      lang = get_resp['objects'][0].get('language')
    else:
      lang = "Resource unavailable"
  else:
    lang = "Remote Notebook so language unidentifiable"
    
  return lang

In [0]:
final_job_data = cleaned_job_data.withColumn('lang',workload_type('notebook_path','source'))
final_job_data = final_job_data.withColumn("workload", f.struct('notebook_path','source','lang')).drop('notebook_path','source','lang')
final_job_data.display()

creator_user_name,job_id,contains_sql_task,workload
prerna.sharma@databricks.com,25422441146053,false,"List(/Users/prerna.sharma@databricks.com/Practice Book/Practice_01, WORKSPACE, PYTHON)"
prerna.sharma@databricks.com,25422441146053,false,"List(/Users/prerna.sharma@databricks.com/2022-05-10 - DBFS Example, WORKSPACE, PYTHON)"
jon.kelley@databricks.com,38678126855960,false,"List(/Users/jon.kelley@databricks.com/Delta Workshop Pepsi/03 Multi-Task Jobs/Task 1, WORKSPACE, PYTHON)"
jon.kelley@databricks.com,38678126855960,false,"List(/Users/jon.kelley@databricks.com/Delta Workshop Pepsi/03 Multi-Task Jobs/Task 2, WORKSPACE, PYTHON)"
jon.kelley@databricks.com,38678126855960,false,"List(/Users/jon.kelley@databricks.com/Delta Workshop Pepsi/03 Multi-Task Jobs/Task 3, WORKSPACE, PYTHON)"
jon.kelley@databricks.com,38678126855960,false,"List(/Users/jon.kelley@databricks.com/Delta Workshop Pepsi/03 Multi-Task Jobs/Task 4, WORKSPACE, PYTHON)"
youssef.mrini@databricks.com,160722975649685,false,"List(/Users/youssef.mrini@databricks.com/Testing, WORKSPACE, SQL)"
sheetal.kushwaha@databricks.com,171379905059617,true,"List(null, null, Remote Notebook so language unidentifiable)"
josh.melton@databricks.com,196245906774103,false,"List(null, null, Remote Notebook so language unidentifiable)"
jaiwant.jonathan@databricks.com,230965078355408,false,"List(/Repos/jaiwant.jonathan@databricks.com/data-engineering-with-databricks/09 - Task Orchestration with Jobs/DE 9.1 - Scheduling Tasks with the Jobs UI/DE 9.1.2 - Reset, WORKSPACE, PYTHON)"


In [0]:
final_job_data.filter("workload.lang = 'SQL' or contains_sql_task = true").groupBy('creator_user_name','job_id','contains_sql_task').agg(f.collect_list('workload').alias('workloads')).display()

creator_user_name,job_id,contains_sql_task,workloads
prerna.sharma@databricks.com,198763332969994,false,"List(List(/Users/prerna.sharma@databricks.com/DNB Aug26/01 - Databricks Workspace and Services/Notebook basics_2, WORKSPACE, SQL))"
nathan.tong@databricks.com,90185521908451,false,"List(List(/Users/nathan.tong@databricks.com/Demos/Delta SQL Rapid Start, WORKSPACE, SQL), List(/Users/nathan.tong@databricks.com/ntong_test, WORKSPACE, SQL))"
shashank.kava@databricks.com,963435679477698,false,"List(List(/Repos/shashank.kava@databricks.com/wflowtest/Click/Match, WORKSPACE, SQL))"
nikit.shah@databricks.com,154471483561685,false,"List(List(/Shared/Tech-Summit-2022/01-Hands-On-Notebooks/1a-DLT-Loan-pipeline-SQL, WORKSPACE, SQL))"
michael.cranford@databricks.com,117909048792713,false,"List(List(/Users/michael.cranford@databricks.com/dlt demo, WORKSPACE, SQL), List(/Users/michael.cranford@databricks.com/Quickstart Notebook (1), WORKSPACE, SQL))"
sheetal.kushwaha@databricks.com,171379905059617,true,"List(List(null, null, Remote Notebook so language unidentifiable))"
sergio.schena@databricks.com,1079514002898377,true,"List(List(/Users/sergio.schena@databricks.com/Rapid Starts/Intro to Databricks/Workflows/0-Setup, WORKSPACE, SQL), List(/Users/sergio.schena@databricks.com/Rapid Starts/Intro to Databricks/Workflows/1A - Gym Data, WORKSPACE, SQL), List(/Users/sergio.schena@databricks.com/Rapid Starts/Intro to Databricks/Workflows/1B - Retail Data, WORKSPACE, SQL))"
sheetal.kushwaha@databricks.com,224807062860319,true,"List(List(null, null, Remote Notebook so language unidentifiable))"
tj.cycyota@databricks.com,814426410866895,true,"List(List(/Table-ACL-Backup, WORKSPACE, PYTHON), List(/SYNC Alternative, WORKSPACE, SCALA))"
rohit.dashora@databricks.com,467983649258141,false,"List(List(/Users/rohit.dashora@databricks.com/Databricks Delta Quickstart (SQL), WORKSPACE, SQL))"


Repos: `file:/Workspace/Repos`

DBFS: `file:/dbfs/FileStore`